# Crack Detection using Drone Images

This notebook processes images to detect cracks using various image processing techniques.

In [1]:
!pip install opencv-python-headless pillow matplotlib scikit-image tensorflow scikit-learn sqlite3

In [2]:
import cv2
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import sqlite3

In [3]:
# Load image
image_path = 'download.jpeg'  # Adjust the image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [4]:
# Noise reduction
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

In [5]:
# Adaptive histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_image = clahe.apply(blurred_image)

In [6]:
# Morphological processing
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(enhanced_image, cv2.MORPH_CLOSE, kernel)

In [7]:
# GLCM
glcm = graycomatrix(enhanced_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
contrast = graycoprops(glcm, 'contrast')[0, 0]
dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
energy = graycoprops(glcm, 'energy')[0, 0]
correlation = graycoprops(glcm, 'correlation')[0, 0]
ASM = graycoprops(glcm, 'ASM')[0, 0]

print(f'GLCM - Contrast: {contrast}, Dissimilarity: {dissimilarity}, Homogeneity: {homogeneity}, Energy: {energy}, Correlation: {correlation}, ASM: {ASM}')

In [8]:
# LBP
radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(enhanced_image, n_points, radius, method='uniform')

In [9]:
# Edge detection
edges = cv2.Canny(enhanced_image, 100, 200)

In [10]:
# Display images
plt.figure(figsize=(10, 6))
plt.subplot(2, 3, 1), plt.imshow(image, cmap='gray'), plt.title('Original Image')
plt.subplot(2, 3, 2), plt.imshow(blurred_image, cmap='gray'), plt.title('Blurred Image')
plt.subplot(2, 3, 3), plt.imshow(enhanced_image, cmap='gray'), plt.title('Enhanced Image')
plt.subplot(2, 3, 4), plt.imshow(morph_image, cmap='gray'), plt.title('Morphological Image')
plt.subplot(2, 3, 5), plt.imshow(edges, cmap='gray'), plt.title('Edges')
plt.show()

In [11]:
# Prepare data for classifiers
def extract_features(image):
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    ASM = graycoprops(glcm, 'ASM')[0, 0]
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    n_bins = int(lbp.max() + 1)
    lbp_hist, _ = np.histogram(lbp, density=True, bins=n_bins, range=(0, n_bins))
    features = np.hstack([contrast, dissimilarity, homogeneity, energy, correlation, ASM, lbp_hist])
    return features

X = []
y = []

# Assuming we have a dataset folder with subfolders 'crack' and 'no_crack'
dataset_path = 'dataset'
for label in ['crack', 'no_crack']:
    folder_path = os.path.join(dataset_path, label)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            features = extract_features(img)
            X.append(features)
            y.append(label)

X = np.array(X)
y = np.array(y)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [12]:
# SVM Classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print(f'SVM Classifier Accuracy: {svm_accuracy}')

In [13]:
# Neural Network Classifier
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

nn_accuracy = model.evaluate(X_test, y_test)[1]
print(f'Neural Network Classifier Accuracy: {nn_accuracy}')

In [14]:
# SQLite Database for Image Storage and Retrieval
conn = sqlite3.connect('images.db')
cursor = conn.cursor()

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS images
                  (id INTEGER PRIMARY KEY, path TEXT, label TEXT, features BLOB)''')

# Insert data
for img_path, label, feature in zip([os.path.join(folder_path, f) for folder_path in [os.path.join(dataset_path, 'crack'), os.path.join(dataset_path, 'no_crack')] for f in os.listdir(folder_path) if f.endswith('.jpg')], y, X):
    cursor.execute('INSERT INTO images (path, label, features) VALUES (?, ?, ?)', (img_path, label, feature.tobytes()))

conn.commit()
conn.close()